# 4-site code that just prints the values

original code ( didnt change anything). this is for only U=25.0


it will produce an 225 row data set 

In [1]:
# ====== Cluster Mean-Field Bose-Hubbard (2x2 plaquette) ======
# Working code, last touched on 22-Oct-2020

import numpy as np
import math
from math import sqrt, factorial
from scipy.sparse import coo_matrix, csr_matrix, eye
from scipy.sparse.linalg import eigsh   # iterative sparse eigensolver
from scipy.linalg import eigh           # dense solver (not really used here)

# --------------------------------------------------------------
# PRECOMPUTED SQUARE ROOTS OF PRIMES
# Used as "tags" for basis states (unique irrational weights so
# every Fock state can be assigned a unique number).
# This allows BinarySearch to quickly map a new basis state
# (after operator application) back to its index.
#
# OLD CODE: did not need this because with a single site, 
# states were trivially labeled 0,1,2,...,nmax.
# --------------------------------------------------------------
primesqt = np.array([
    1.41421356, 1.73205080, 2.23606797, 2.64575131, 3.31662479,
    3.60555127, 4.12310562, 4.35889894, 4.79583152, 5.38516480,
    5.56776436, 6.08276253, 6.40312423, 6.55743852, 6.85565460,
    7.28010988, 7.68114574, 7.81024967, 8.18535277, 8.42614977,
    8.54400374, 8.88819441, 9.11043357, 9.43398113, 9.84885780,
    10.04987562, 10.14889156, 10.34408043, 10.44030650, 10.63014581,
    11.26942766, 11.44552314, 11.70469991, 11.78982612, 12.20655561,
    12.28820572, 12.52996408, 12.76714533, 12.92284798, 13.15294643,
    13.37908816, 13.45362404, 13.82027496, 13.89244398, 14.03566884,
    14.10673597, 14.52583904, 14.93318452, 15.06651917, 15.13274595
])

# --------------------------------------------------------------
# PHYSICAL PARAMETERS
# --------------------------------------------------------------
tx = 1.0   # hopping in x
ty = 1.0   # hopping in y
U  = 25.0  # on-site interaction strength

# chemical potential scan
mu_min, mu_max, dmu = -5.0, 40.0, 0.2

# cluster geometry (2x2 plaquette)
xlen, ylen = 2, 2
m = xlen * ylen   # total number of sites = 4

nmax = 4          # maximum occupation per site

# Hilbert space dimension:
# OLD CODE: dim = nmax+1
# NEW CODE: dim = (nmax+1)^m, exponential in #sites
dim = (nmax+1)**m

# --------------------------------------------------------------
# BASIS GENERATION
# Generate all boson configurations (n1,n2,n3,n4) with 0..nmax
# --------------------------------------------------------------
def basis_gen(n, k, min_elem, max_elem):
    allowed = range(max_elem, min_elem-1, -1)
    def helper(n, k, t):
        if k == 0:
            if n == 0:
                yield t
        elif k == 1:
            if n in allowed:
                yield t + (n,)
        elif min_elem * k <= n <= max_elem * k:
            for v in allowed:
                yield from helper(n - v, k - 1, t + (v,))
    return helper(n, k, ())

# --------------------------------------------------------------
# Binary search lookup into sorted tag list (see primesqt trick)
# --------------------------------------------------------------
def BinarySearch(lys, val):  
    first, last, index = 0, len(lys)-1, -1
    while (first <= last) and (index == -1):
        mid = (first+last)//2
        if lys[mid] == val:
            index = mid
        elif val < lys[mid]:
            last = mid -1
        else:
            first = mid +1
    return index

# --------------------------------------------------------------
# LATTICE STRUCTURE: 2x2 PLAQUETTE
# lattice[y][x] = site index (0..3)
# --------------------------------------------------------------
lattice = np.zeros((ylen,xlen),int)
lat_ind = 0
for yy in range(ylen):
    for xx in range(xlen):
        lattice[yy][xx] = lat_ind
        lat_ind += 1

# nearest-neighbor pairs along rows
row_pair = []
for yy in range(ylen):
    for xx in range(xlen-1):
        row_pair.append([lattice[yy][xx], lattice[yy][xx+1]])

# nearest-neighbor pairs along columns
col_pair = []
for xx in range(xlen):
    for yy in range(ylen-1):
        col_pair.append([lattice[yy][xx], lattice[yy+1][xx]])

# --------------------------------------------------------------
# BOUNDARY SITES
# Identify boundary sites (left/right/top/bottom edges).
# These couple to the external mean field.
# OLD CODE: trivial, single site always boundary.
# --------------------------------------------------------------
lbs = [xlen * x for x in range(0, ylen)]                 # left boundary sites
rbs = [(xlen * (x+1))-1 for x in range(0, ylen)]         # right boundary
tbs = [x for x in range(lbs[0], rbs[0]+1)]               # top row
bbs = [x for x in range(lbs[-1], rbs[-1]+1)]             # bottom row
all_bs = list(set().union(lbs, rbs, tbs, bbs))           # all unique boundary sites
lr_zip = list(zip(lbs, rbs))  # left-right pairs
tb_zip = list(zip(tbs, bbs))  # top-bottom pairs
bds = lr_zip + tb_zip         # all boundary pairs

nds = len(all_bs)             # number of boundary sites

# --------------------------------------------------------------
# BASIS CONSTRUCTION
# 'a' = list of all basis states (tuples of occupancies)
# e.g., for m=2, nmax=2: (0,0),(0,1),(0,2),(1,0),...
# --------------------------------------------------------------
a = []
for n in range(0, (nmax*m)+1):
    for x in basis_gen(n, m, 0, nmax):
        a.append(x)
bscount = len(a)

# tag each basis state with unique number using primesqt
T = np.zeros(dim)
for ii in range(dim):
    for jj in range(m):
        T[ii] += primesqt[jj]*float(a[ii][jj])

ind = np.argsort(T)  # index array for sorting
T   = sorted(T)

# --------------------------------------------------------------
# OPERATOR MATRICES
# Build annihilation (a) and number (n) operators for each site.
# OLD CODE: only one operator (single site).
# NEW CODE: one per site in cluster.
# --------------------------------------------------------------

# --- Annihilation operators (a) ---
a_mat_dic = {}
for ii in all_bs:  # only for boundary sites (needed for φ update)
    row, col, val = [], [], []
    for vv in range(dim):
        Tg = 0
        if a[vv][ii]+1 < nmax+1:   # can create one more boson
            ket = np.copy(a[vv])
            aiD = sqrt(float(ket[ii]+1.0))
            ket[ii] = ket[ii]+1
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu)
            col.append(vv)
            val.append(aiD)
    # construct sparse matrix for a† (hence getH() at the end)
    a_mat = (coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()).getH()
    a_mat_dic[ii] = a_mat

# --- Number operators (n) ---
n_mat_dic = {}
for ii in range(m):
    row, col, val = [], [], []
    for vv in range(dim):
        if a[vv][ii] > 0:
            ket = np.copy(a[vv])
            diag = float(ket[ii])
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu)
            col.append(vv)
            val.append(diag)
    n_mat = coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()
    n_mat_dic[ii] = n_mat

# --------------------------------------------------------------
# HOPPING TERM (exact inside cluster)
# Unlike old code, where hopping was only mean-field decoupled,
# here intra-cluster hopping is treated exactly.
# --------------------------------------------------------------
row, col, val = [], [], []
# row-wise nearest neighbor hopping
for ii in row_pair:
    for vv in range(dim):
        if a[vv][ii[1]] >= 1.0 and a[vv][ii[0]]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiDaj = -tx*sqrt(float((ket[ii[0]]+1.0)*ket[ii[1]]))
            ket[ii[0]] += 1
            ket[ii[1]] -= 1
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu)
            col.append(vv)
            val.append(aiDaj)

# col-wise nearest neighbor hopping
for ii in col_pair:
    for vv in range(dim):
        if a[vv][ii[1]] >= 1.0 and a[vv][ii[0]]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiDaj = -ty*sqrt(float((ket[ii[0]]+1.0)*ket[ii[1]]))
            ket[ii[0]] += 1
            ket[ii[1]] -= 1
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu)
            col.append(vv)
            val.append(aiDaj)

hop_mat = coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()
hop_mat = hop_mat + hop_mat.getH()  # Hermitian

# --------------------------------------------------------------
# SELF-CONSISTENT MEAN FIELD LOOP
# OLD CODE: only one φ (global order parameter).
# NEW CODE: one φ per boundary site.
# --------------------------------------------------------------

phi, phi_new, diff, accuracy = {}, {}, {}, {}
gs = np.zeros(dim, dtype='float64')

for bsi in all_bs:
    accuracy[bsi] = 1e-6   # convergence tolerance

phi_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()
psq_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()
int_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()

for mu in np.arange(mu_min, mu_max, dmu):
    rho = {}
    iteration = 0
    for bsi in all_bs:
        phi[bsi] = 0.0
        phi_new[bsi] = 0.3   # initial guess
        diff[bsi] = abs(phi[bsi] - phi_new[bsi])

    # iterate until all boundary φ’s converge
    while all(diff[bsi] > accuracy[bsi] for bsi in all_bs):
        H_mat = 0.0
        phi_mat = 0.0
        psq_mat = 0.0
        int_mat = 0.0

        for bsi in all_bs:
            phi[bsi] = phi_new[bsi]

        # mean-field boundary coupling terms
        for xx in lr_zip:
            phi_mat += - tx*phi[xx[0]]*(a_mat_dic[xx[1]].getH() + a_mat_dic[xx[1]])
            phi_mat += - tx*phi[xx[1]]*(a_mat_dic[xx[0]].getH() + a_mat_dic[xx[0]])
            psq_mat += + 2.0*tx*phi[xx[0]]*phi[xx[1]]*eye(dim, dtype=np.double, format='csr')
        for yy in tb_zip:
            phi_mat += - ty*phi[yy[0]]*(a_mat_dic[yy[1]].getH() + a_mat_dic[yy[1]])
            phi_mat += - ty*phi[yy[1]]*(a_mat_dic[yy[0]].getH() + a_mat_dic[yy[0]])
            psq_mat += + 2.0*ty*phi[yy[0]]*phi[yy[1]]*eye(dim, dtype=np.double, format='csr')

        # onsite interaction + chemical potential
        for ii in range(m):
            int_mat += (U/2.0)*n_mat_dic[ii]*(n_mat_dic[ii]-eye(dim, dtype=np.double, format='csr'))
            int_mat += - mu*n_mat_dic[ii]

        # full Hamiltonian = hopping + onsite + mean-field
        H_mat = hop_mat + int_mat + psq_mat + phi_mat

        # diagonalize (smallest eigenvalues)
        nev = min(dim,3)
        if nev == 1:
            eigenvalues = H_mat[0,0]
            eigenvectors = eye(dim, dtype=np.double, format='csr')
        else:
            eigenvalues, eigenvectors = eigsh(H_mat, k=3, which='SA')

        gs = eigenvectors[:,0]  # ground state

        # update φ’s (self-consistency condition)
        for bsi in all_bs:
            phi_new[bsi] = gs.transpose().dot(a_mat_dic[bsi].dot(gs))
            diff[bsi] = abs(phi[bsi] - phi_new[bsi])

        iteration += 1
        if iteration > 10000:  # safety break
            break

    # after convergence: compute densities ρ_i
    for bsi in all_bs:
        rho[bsi] = gs.transpose().dot(n_mat_dic[bsi].dot(gs))

    # print results (densities, φ², energy, iterations)
    print("{0:3.2f} {1:6.2f} {2:6.2f} {3:8.4f} {4:8.4f} {5:8.4f} {6:8.4f} "
          "{7:8.4f} {8:8.4f} {9:8.4f} {10:8.4f} {11:12.6f} {12:6d}"
        .format(tx, U, mu,
                rho[0], rho[1], rho[2], rho[3],
                phi_new[0]**2, phi_new[1]**2, phi_new[2]**2, phi_new[3]**2,
                eigenvalues[0], iteration))


1.00  25.00  -5.00   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     30
1.00  25.00  -4.80   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     35
1.00  25.00  -4.60   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     43
1.00  25.00  -4.40   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     59
1.00  25.00  -4.20   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000    103
1.00  25.00  -4.00   0.0001   0.0001   0.0001   0.0001   0.0001   0.0001   0.0001   0.0001     0.000000   3596
1.00  25.00  -3.80   0.0366   0.0366   0.0366   0.0366   0.0356   0.0356   0.0356   0.0356    -0.014704     48
1.00  25.00  -3.60   0.0719   0.0719   0.0719   0.0719   0.0682   0.0682   0.0682   0.0682    -0.058174     25
1.00  25.00  -3.40   0.1062   0.1062   0.1062   0.1062   0.0979   0.0979   0.0979   0.0979    -0.129480     16
1

# Saving as csv file  including the mu_max value too ( so 226 datapoints will be there)
made sure the values in the csv rounded off same as the original code

In [2]:
# ====== Cluster Mean-Field Bose-Hubbard (2x2 plaquette) ======
# Working code, last touched on 22-Oct-2020

import numpy as np
import math
from math import sqrt, factorial
from scipy.sparse import coo_matrix, csr_matrix, eye
from scipy.sparse.linalg import eigsh   # iterative sparse eigensolver
from scipy.linalg import eigh           # dense solver (not really used here)

# >>> storage additions >>>
import csv
from pathlib import Path
# <<< storage additions <<<

# --------------------------------------------------------------
# PRECOMPUTED SQUARE ROOTS OF PRIMES
# --------------------------------------------------------------
primesqt = np.array([
    1.41421356, 1.73205080, 2.23606797, 2.64575131, 3.31662479,
    3.60555127, 4.12310562, 4.35889894, 4.79583152, 5.38516480,
    5.56776436, 6.08276253, 6.40312423, 6.55743852, 6.85565460,
    7.28010988, 7.68114574, 7.81024967, 8.18535277, 8.42614977,
    8.54400374, 8.88819441, 9.11043357, 9.43398113, 9.84885780,
    10.04987562, 10.14889156, 10.34408043, 10.44030650, 10.63014581,
    11.26942766, 11.44552314, 11.70469991, 11.78982612, 12.20655561,
    12.28820572, 12.52996408, 12.76714533, 12.92284798, 13.15294643,
    13.37908816, 13.45362404, 13.82027496, 13.89244398, 14.03566884,
    14.10673597, 14.52583904, 14.93318452, 15.06651917, 15.13274595
])

# --------------------------------------------------------------
# PHYSICAL PARAMETERS
# --------------------------------------------------------------
tx = 1.0   # hopping in x
ty = 1.0   # hopping in y
U  = 25.0  # on-site interaction strength

# chemical potential scan
mu_min, mu_max, dmu = -5.0, 40.0, 0.2

# cluster geometry (2x2 plaquette)
xlen, ylen = 2, 2
m = xlen * ylen   # total number of sites = 4

nmax = 4          # maximum occupation per site

# Hilbert space dimension:
dim = (nmax+1)**m

# --------------------------------------------------------------
# BASIS GENERATION
# --------------------------------------------------------------
def basis_gen(n, k, min_elem, max_elem):
    allowed = range(max_elem, min_elem-1, -1)
    def helper(n, k, t):
        if k == 0:
            if n == 0:
                yield t
        elif k == 1:
            if n in allowed:
                yield t + (n,)
        elif min_elem * k <= n <= max_elem * k:
            for v in allowed:
                yield from helper(n - v, k - 1, t + (v,))
    return helper(n, k, ())

# --------------------------------------------------------------
# Binary search lookup
# --------------------------------------------------------------
def BinarySearch(lys, val):  
    first, last, index = 0, len(lys)-1, -1
    while (first <= last) and (index == -1):
        mid = (first+last)//2
        if lys[mid] == val:
            index = mid
        elif val < lys[mid]:
            last = mid -1
        else:
            first = mid +1
    return index

# --------------------------------------------------------------
# LATTICE STRUCTURE: 2x2 PLAQUETTE
# --------------------------------------------------------------
lattice = np.zeros((ylen,xlen),int)
lat_ind = 0
for yy in range(ylen):
    for xx in range(xlen):
        lattice[yy][xx] = lat_ind
        lat_ind += 1

row_pair = []
for yy in range(ylen):
    for xx in range(xlen-1):
        row_pair.append([lattice[yy][xx], lattice[yy][xx+1]])

col_pair = []
for xx in range(xlen):
    for yy in range(ylen-1):
        col_pair.append([lattice[yy][xx], lattice[yy+1][xx]])

# --------------------------------------------------------------
# BOUNDARY SITES
# --------------------------------------------------------------
lbs = [xlen * x for x in range(0, ylen)]
rbs = [(xlen * (x+1))-1 for x in range(0, ylen)]
tbs = [x for x in range(lbs[0], rbs[0]+1)]
bbs = [x for x in range(lbs[-1], rbs[-1]+1)]
all_bs = list(set().union(lbs, rbs, tbs, bbs))
lr_zip = list(zip(lbs, rbs))
tb_zip = list(zip(tbs, bbs))
bds = lr_zip + tb_zip
nds = len(all_bs)

# --------------------------------------------------------------
# BASIS CONSTRUCTION
# --------------------------------------------------------------
a = []
for n in range(0, (nmax*m)+1):
    for x in basis_gen(n, m, 0, nmax):
        a.append(x)
bscount = len(a)

T = np.zeros(dim)
for ii in range(dim):
    for jj in range(m):
        T[ii] += primesqt[jj]*float(a[ii][jj])

ind = np.argsort(T)
T   = sorted(T)

# --------------------------------------------------------------
# OPERATOR MATRICES
# --------------------------------------------------------------
a_mat_dic = {}
for ii in all_bs:
    row, col, val = [], [], []
    for vv in range(dim):
        Tg = 0
        if a[vv][ii]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiD = sqrt(float(ket[ii]+1.0))
            ket[ii] = ket[ii]+1
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiD)
    a_mat = (coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()).getH()
    a_mat_dic[ii] = a_mat

n_mat_dic = {}
for ii in range(m):
    row, col, val = [], [], []
    for vv in range(dim):
        if a[vv][ii] > 0:
            ket = np.copy(a[vv])
            diag = float(ket[ii])
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(diag)
    n_mat = coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()
    n_mat_dic[ii] = n_mat

# --------------------------------------------------------------
# HOPPING TERM (exact inside cluster)
# --------------------------------------------------------------
row, col, val = [], [], []
for ii in row_pair:
    for vv in range(dim):
        if a[vv][ii[1]] >= 1.0 and a[vv][ii[0]]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiDaj = -tx*sqrt(float((ket[ii[0]]+1.0)*ket[ii[1]]))
            ket[ii[0]] += 1; ket[ii[1]] -= 1
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiDaj)

for ii in col_pair:
    for vv in range(dim):
        if a[vv][ii[1]] >= 1.0 and a[vv][ii[0]]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiDaj = -ty*sqrt(float((ket[ii[0]]+1.0)*ket[ii[1]]))
            ket[ii[0]] += 1; ket[ii[1]] -= 1
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiDaj)

hop_mat = coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()
hop_mat = hop_mat + hop_mat.getH()

# --------------------------------------------------------------
# SELF-CONSISTENT MEAN FIELD LOOP
# --------------------------------------------------------------
phi, phi_new, diff, accuracy = {}, {}, {}, {}
gs = np.zeros(dim, dtype='float64')
for bsi in all_bs:
    accuracy[bsi] = 1e-6

phi_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()
psq_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()
int_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()

# >>> storage additions >>>
results = []
# <<< storage additions <<<

for mu in np.arange(mu_min, mu_max+dmu, dmu):
    rho = {}
    iteration = 0
    for bsi in all_bs:
        phi[bsi] = 0.0
        phi_new[bsi] = 0.3
        diff[bsi] = abs(phi[bsi] - phi_new[bsi])

    while all(diff[bsi] > accuracy[bsi] for bsi in all_bs):
        H_mat = 0.0
        phi_mat = 0.0
        psq_mat = 0.0
        int_mat = 0.0

        for bsi in all_bs:
            phi[bsi] = phi_new[bsi]

        for xx in lr_zip:
            phi_mat += - tx*phi[xx[0]]*(a_mat_dic[xx[1]].getH() + a_mat_dic[xx[1]])
            phi_mat += - tx*phi[xx[1]]*(a_mat_dic[xx[0]].getH() + a_mat_dic[xx[0]])
            psq_mat += + 2.0*tx*phi[xx[0]]*phi[xx[1]]*eye(dim, dtype=np.double, format='csr')
        for yy in tb_zip:
            phi_mat += - ty*phi[yy[0]]*(a_mat_dic[yy[1]].getH() + a_mat_dic[yy[1]])
            phi_mat += - ty*phi[yy[1]]*(a_mat_dic[yy[0]].getH() + a_mat_dic[yy[0]])
            psq_mat += + 2.0*ty*phi[yy[0]]*phi[yy[1]]*eye(dim, dtype=np.double, format='csr')

        for ii in range(m):
            int_mat += (U/2.0)*n_mat_dic[ii]*(n_mat_dic[ii]-eye(dim, dtype=np.double, format='csr'))
            int_mat += - mu*n_mat_dic[ii]

        H_mat = hop_mat + int_mat + psq_mat + phi_mat

        nev = min(dim,3)
        if nev == 1:
            eigenvalues = H_mat[0,0]
            eigenvectors = eye(dim, dtype=np.double, format='csr')
        else:
            eigenvalues, eigenvectors = eigsh(H_mat, k=3, which='SA')

        gs = eigenvectors[:,0]

        for bsi in all_bs:
            phi_new[bsi] = gs.transpose().dot(a_mat_dic[bsi].dot(gs))
            diff[bsi] = abs(phi[bsi] - phi_new[bsi])

        iteration += 1
        if iteration > 10000:
            break

    for bsi in all_bs:
        rho[bsi] = gs.transpose().dot(n_mat_dic[bsi].dot(gs))

    print("{0:3.2f} {1:6.2f} {2:6.2f} {3:8.4f} {4:8.4f} {5:8.4f} {6:8.4f} "
          "{7:8.4f} {8:8.4f} {9:8.4f} {10:8.4f} {11:12.6f} {12:6d}"
        .format(tx, U, mu,
                rho[0], rho[1], rho[2], rho[3],
                phi_new[0]**2, phi_new[1]**2, phi_new[2]**2, phi_new[3]**2,
                eigenvalues[0], iteration))
    # the csv file  prints out the rounded number but it will contains proper floats for later analysis
    results.append([
        round(tx,2), round(U,2), round(mu,2),
        round(rho[0],4), round(rho[1],4), round(rho[2],4), round(rho[3],4),
        round(phi_new[0]**2,4), round(phi_new[1]**2,4), round(phi_new[2]**2,4), round(phi_new[3]**2,4),
        round(float(eigenvalues[0]),6), int(iteration) ])
    # This method stores without  rounding off the decimals
    # # >>> storage additions >>>
    # results.append([
    #     tx, U, mu,
    #     float(rho[0]), float(rho[1]), float(rho[2]), float(rho[3]),
    #     float(phi_new[0]**2), float(phi_new[1]**2), float(phi_new[2]**2), float(phi_new[3]**2),
    #     float(eigenvalues[0]), int(iteration)
    # ])
    # # <<< storage additions <<<


   # rounding off decimals in the same way as the print exactly 
   # >>> storage additions >>>
    # results.append([
    #     f"{tx:3.2f}", f"{U:6.2f}", f"{mu:6.2f}",
    #     f"{rho[0]:8.4f}", f"{rho[1]:8.4f}", f"{rho[2]:8.4f}", f"{rho[3]:8.4f}",
    #     f"{phi_new[0]**2:8.4f}", f"{phi_new[1]**2:8.4f}", f"{phi_new[2]**2:8.4f}", f"{phi_new[3]**2:8.4f}",
    #     f"{eigenvalues[0]:12.6f}", f"{iteration:6d}"
    # ])
    # <<< storage additions <<<



   

   
# >>> storage additions >>>
# Save CSV mirroring the print layout
headers = [
    "tx","U","mu",
    "rho0","rho1","rho2","rho3",
    "phi0_sq","phi1_sq","phi2_sq","phi3_sq",
    "E0","iters"
]
out_csv = Path("results_savingcsv_1.csv")
with out_csv.open("w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(results)

# Also save as a compact NumPy bundle for fast reload
np.savez_compressed(
    "results_savingnpz_1.npz",
    headers=np.array(headers, dtype=object),
    data=np.array(results, dtype=float)
)

# <<< storage additions <<<


1.00  25.00  -5.00   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     30
1.00  25.00  -4.80   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     35
1.00  25.00  -4.60   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     43
1.00  25.00  -4.40   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     59
1.00  25.00  -4.20   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000    103
1.00  25.00  -4.00   0.0001   0.0001   0.0001   0.0001   0.0001   0.0001   0.0001   0.0001     0.000000   3596
1.00  25.00  -3.80   0.0366   0.0366   0.0366   0.0366   0.0356   0.0356   0.0356   0.0356    -0.014704     48
1.00  25.00  -3.60   0.0719   0.0719   0.0719   0.0719   0.0682   0.0682   0.0682   0.0682    -0.058174     25
1.00  25.00  -3.40   0.1062   0.1062   0.1062   0.1062   0.0979   0.0979   0.0979   0.0979    -0.129480     16
1

# Added a U sweep 
results are stored as results_2.csv

In [3]:
# ====== Cluster Mean-Field Bose-Hubbard (2x2 plaquette) ======
# Working code, last touched on 22-Oct-2020

import numpy as np
import math
from math import sqrt, factorial
from scipy.sparse import coo_matrix, csr_matrix, eye
from scipy.sparse.linalg import eigsh   # iterative sparse eigensolver
from scipy.linalg import eigh           # dense solver (not really used here)

# >>> storage additions >>>
import csv
from pathlib import Path
# <<< storage additions <<<

# --------------------------------------------------------------
# PRECOMPUTED SQUARE ROOTS OF PRIMES
# --------------------------------------------------------------
primesqt = np.array([
    1.41421356, 1.73205080, 2.23606797, 2.64575131, 3.31662479,
    3.60555127, 4.12310562, 4.35889894, 4.79583152, 5.38516480,
    5.56776436, 6.08276253, 6.40312423, 6.55743852, 6.85565460,
    7.28010988, 7.68114574, 7.81024967, 8.18535277, 8.42614977,
    8.54400374, 8.88819441, 9.11043357, 9.43398113, 9.84885780,
    10.04987562, 10.14889156, 10.34408043, 10.44030650, 10.63014581,
    11.26942766, 11.44552314, 11.70469991, 11.78982612, 12.20655561,
    12.28820572, 12.52996408, 12.76714533, 12.92284798, 13.15294643,
    13.37908816, 13.45362404, 13.82027496, 13.89244398, 14.03566884,
    14.10673597, 14.52583904, 14.93318452, 15.06651917, 15.13274595
])

# --------------------------------------------------------------
# PHYSICAL PARAMETERS
# --------------------------------------------------------------
tx = 1.0   # hopping in x
ty = 1.0   # hopping in y
# U  = 25.0  # on-site interaction strength

# Instead of fixed U, we define a sweep
U_min, U_max, dU = 0.0, 25.0, 1.0   # <-- choose your U range + step

# chemical potential scan
mu_min, mu_max, dmu = -5.0, 40.0, 0.2

# cluster geometry (2x2 plaquette)
xlen, ylen = 2, 2
m = xlen * ylen   # total number of sites = 4

nmax = 4          # maximum occupation per site

# Hilbert space dimension:
dim = (nmax+1)**m

# --------------------------------------------------------------
# BASIS GENERATION
# --------------------------------------------------------------
def basis_gen(n, k, min_elem, max_elem):
    allowed = range(max_elem, min_elem-1, -1)
    def helper(n, k, t):
        if k == 0:
            if n == 0:
                yield t
        elif k == 1:
            if n in allowed:
                yield t + (n,)
        elif min_elem * k <= n <= max_elem * k:
            for v in allowed:
                yield from helper(n - v, k - 1, t + (v,))
    return helper(n, k, ())

# --------------------------------------------------------------
# Binary search lookup
# --------------------------------------------------------------
def BinarySearch(lys, val):  
    first, last, index = 0, len(lys)-1, -1
    while (first <= last) and (index == -1):
        mid = (first+last)//2
        if lys[mid] == val:
            index = mid
        elif val < lys[mid]:
            last = mid -1
        else:
            first = mid +1
    return index

# --------------------------------------------------------------
# LATTICE STRUCTURE: 2x2 PLAQUETTE
# --------------------------------------------------------------
lattice = np.zeros((ylen,xlen),int)
lat_ind = 0
for yy in range(ylen):
    for xx in range(xlen):
        lattice[yy][xx] = lat_ind
        lat_ind += 1

row_pair = []
for yy in range(ylen):
    for xx in range(xlen-1):
        row_pair.append([lattice[yy][xx], lattice[yy][xx+1]])

col_pair = []
for xx in range(xlen):
    for yy in range(ylen-1):
        col_pair.append([lattice[yy][xx], lattice[yy+1][xx]])

# --------------------------------------------------------------
# BOUNDARY SITES
# --------------------------------------------------------------
lbs = [xlen * x for x in range(0, ylen)]
rbs = [(xlen * (x+1))-1 for x in range(0, ylen)]
tbs = [x for x in range(lbs[0], rbs[0]+1)]
bbs = [x for x in range(lbs[-1], rbs[-1]+1)]
all_bs = list(set().union(lbs, rbs, tbs, bbs))
lr_zip = list(zip(lbs, rbs))
tb_zip = list(zip(tbs, bbs))
bds = lr_zip + tb_zip
nds = len(all_bs)

# --------------------------------------------------------------
# BASIS CONSTRUCTION
# --------------------------------------------------------------
a = []
for n in range(0, (nmax*m)+1):
    for x in basis_gen(n, m, 0, nmax):
        a.append(x)
bscount = len(a)

T = np.zeros(dim)
for ii in range(dim):
    for jj in range(m):
        T[ii] += primesqt[jj]*float(a[ii][jj])

ind = np.argsort(T)
T   = sorted(T)

# --------------------------------------------------------------
# OPERATOR MATRICES
# --------------------------------------------------------------
a_mat_dic = {}
for ii in all_bs:
    row, col, val = [], [], []
    for vv in range(dim):
        Tg = 0
        if a[vv][ii]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiD = sqrt(float(ket[ii]+1.0))
            ket[ii] = ket[ii]+1
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiD)
    a_mat = (coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()).getH()
    a_mat_dic[ii] = a_mat

n_mat_dic = {}
for ii in range(m):
    row, col, val = [], [], []
    for vv in range(dim):
        if a[vv][ii] > 0:
            ket = np.copy(a[vv])
            diag = float(ket[ii])
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(diag)
    n_mat = coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()
    n_mat_dic[ii] = n_mat

# --------------------------------------------------------------
# HOPPING TERM (exact inside cluster)
# --------------------------------------------------------------
row, col, val = [], [], []
for ii in row_pair:
    for vv in range(dim):
        if a[vv][ii[1]] >= 1.0 and a[vv][ii[0]]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiDaj = -tx*sqrt(float((ket[ii[0]]+1.0)*ket[ii[1]]))
            ket[ii[0]] += 1; ket[ii[1]] -= 1
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiDaj)

for ii in col_pair:
    for vv in range(dim):
        if a[vv][ii[1]] >= 1.0 and a[vv][ii[0]]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiDaj = -ty*sqrt(float((ket[ii[0]]+1.0)*ket[ii[1]]))
            ket[ii[0]] += 1; ket[ii[1]] -= 1
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiDaj)

hop_mat = coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()
hop_mat = hop_mat + hop_mat.getH()

# --------------------------------------------------------------
# SELF-CONSISTENT MEAN FIELD LOOP
# --------------------------------------------------------------
phi, phi_new, diff, accuracy = {}, {}, {}, {}
gs = np.zeros(dim, dtype='float64')
for bsi in all_bs:
    accuracy[bsi] = 1e-6

phi_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()
psq_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()
int_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()

# >>> storage additions >>>
results = []
# <<< storage additions <<<

#np.arange(start, stop, step) goes up to but not including stop.
#So if you write

for U in np.arange(U_min, U_max+dU, dU):
    for mu in np.arange(mu_min, mu_max+dmu, dmu):
        rho = {}
        iteration = 0
        for bsi in all_bs:
            phi[bsi] = 0.0
            phi_new[bsi] = 0.3
            diff[bsi] = abs(phi[bsi] - phi_new[bsi])
    
        while all(diff[bsi] > accuracy[bsi] for bsi in all_bs):
            H_mat = 0.0
            phi_mat = 0.0
            psq_mat = 0.0
            int_mat = 0.0
    
            for bsi in all_bs:
                phi[bsi] = phi_new[bsi]
    
            for xx in lr_zip:
                phi_mat += - tx*phi[xx[0]]*(a_mat_dic[xx[1]].getH() + a_mat_dic[xx[1]])
                phi_mat += - tx*phi[xx[1]]*(a_mat_dic[xx[0]].getH() + a_mat_dic[xx[0]])
                psq_mat += + 2.0*tx*phi[xx[0]]*phi[xx[1]]*eye(dim, dtype=np.double, format='csr')
            for yy in tb_zip:
                phi_mat += - ty*phi[yy[0]]*(a_mat_dic[yy[1]].getH() + a_mat_dic[yy[1]])
                phi_mat += - ty*phi[yy[1]]*(a_mat_dic[yy[0]].getH() + a_mat_dic[yy[0]])
                psq_mat += + 2.0*ty*phi[yy[0]]*phi[yy[1]]*eye(dim, dtype=np.double, format='csr')
    
            for ii in range(m):
                int_mat += (U/2.0)*n_mat_dic[ii]*(n_mat_dic[ii]-eye(dim, dtype=np.double, format='csr'))
                int_mat += - mu*n_mat_dic[ii]
    
            H_mat = hop_mat + int_mat + psq_mat + phi_mat
    
            nev = min(dim,3)
            if nev == 1:
                eigenvalues = H_mat[0,0]
                eigenvectors = eye(dim, dtype=np.double, format='csr')
            else:
                eigenvalues, eigenvectors = eigsh(H_mat, k=3, which='SA')
    
            gs = eigenvectors[:,0]
    
            for bsi in all_bs:
                phi_new[bsi] = gs.transpose().dot(a_mat_dic[bsi].dot(gs))
                diff[bsi] = abs(phi[bsi] - phi_new[bsi])
    
            iteration += 1
            if iteration > 10000:
                break
    
        for bsi in all_bs:
            rho[bsi] = gs.transpose().dot(n_mat_dic[bsi].dot(gs))
    
        print("{0:3.2f} {1:6.2f} {2:6.2f} {3:8.4f} {4:8.4f} {5:8.4f} {6:8.4f} "
              "{7:8.4f} {8:8.4f} {9:8.4f} {10:8.4f} {11:12.6f} {12:6d}"
            .format(tx, U, mu,
                    rho[0], rho[1], rho[2], rho[3],
                    phi_new[0]**2, phi_new[1]**2, phi_new[2]**2, phi_new[3]**2,
                    eigenvalues[0], iteration))
        # the csv file  prints out the rounded number but it will contains proper floats for later analysis
        results.append([
            round(tx,2), round(U,2), round(mu,2),
            round(rho[0],4), round(rho[1],4), round(rho[2],4), round(rho[3],4),
            round(phi_new[0]**2,4), round(phi_new[1]**2,4), round(phi_new[2]**2,4), round(phi_new[3]**2,4),
            round(float(eigenvalues[0]),6), int(iteration) ])
    # This method stores without  rounding off the decimals
    # # >>> storage additions >>>
    # results.append([
    #     tx, U, mu,
    #     float(rho[0]), float(rho[1]), float(rho[2]), float(rho[3]),
    #     float(phi_new[0]**2), float(phi_new[1]**2), float(phi_new[2]**2), float(phi_new[3]**2),
    #     float(eigenvalues[0]), int(iteration)
    # ])
    # # <<< storage additions <<<


   # rounding off decimals in the same way as the print exactly 
   # >>> storage additions >>>
    # results.append([
    #     f"{tx:3.2f}", f"{U:6.2f}", f"{mu:6.2f}",
    #     f"{rho[0]:8.4f}", f"{rho[1]:8.4f}", f"{rho[2]:8.4f}", f"{rho[3]:8.4f}",
    #     f"{phi_new[0]**2:8.4f}", f"{phi_new[1]**2:8.4f}", f"{phi_new[2]**2:8.4f}", f"{phi_new[3]**2:8.4f}",
    #     f"{eigenvalues[0]:12.6f}", f"{iteration:6d}"
    # ])
    # <<< storage additions <<<



   

   
# >>> storage additions >>>
# Save CSV mirroring the print layout
headers = [
    "tx","U","mu",
    "rho0","rho1","rho2","rho3",
    "phi0_sq","phi1_sq","phi2_sq","phi3_sq",
    "E0","iters"
]
out_csv = Path("results_2.csv")
with out_csv.open("w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(results)

# Also save as a compact NumPy bundle for fast reload
np.savez_compressed(
    "results_2.npz",
    headers=np.array(headers, dtype=object),
    data=np.array(results, dtype=float)
)

# <<< storage additions <<<


1.00   0.00  -5.00   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     30
1.00   0.00  -4.80   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     35
1.00   0.00  -4.60   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     44
1.00   0.00  -4.40   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000     61
1.00   0.00  -4.20   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000     0.000000    109
1.00   0.00  -4.00   0.0690   0.0690   0.0690   0.0690   0.0690   0.0690   0.0690   0.0690     0.000001  10001
1.00   0.00  -3.80   1.0032   1.0032   1.0032   1.0032   0.9887   0.9887   0.9887   0.9887    -0.603506     35
1.00   0.00  -3.60   1.2744   1.2744   1.2744   1.2744   1.2367   1.2367   1.2367   1.2367    -1.522379     21
1.00   0.00  -3.40   1.4671   1.4671   1.4671   1.4671   1.4020   1.4020   1.4020   1.4020    -2.622406     15
1

# Added U sweeping Range and File name , Parallel Job handling for genertaing large dataset

In [4]:
# ====== Cluster Mean-Field Bose-Hubbard (2x2 plaquette) ======
# Working code, last touched on 22-Oct-2020

import numpy as np
import math
from math import sqrt, factorial
from scipy.sparse import coo_matrix, csr_matrix, eye
from scipy.sparse.linalg import eigsh   # iterative sparse eigensolver
from scipy.linalg import eigh           # dense solver (not really used here)

# >>> storage additions >>>
import csv
from pathlib import Path
from joblib import Parallel, delayed
import os

# <<< storage additions <<<

# --------------------------------------------------------------
# PRECOMPUTED SQUARE ROOTS OF PRIMES
# --------------------------------------------------------------
primesqt = np.array([
    1.41421356, 1.73205080, 2.23606797, 2.64575131, 3.31662479,
    3.60555127, 4.12310562, 4.35889894, 4.79583152, 5.38516480,
    5.56776436, 6.08276253, 6.40312423, 6.55743852, 6.85565460,
    7.28010988, 7.68114574, 7.81024967, 8.18535277, 8.42614977,
    8.54400374, 8.88819441, 9.11043357, 9.43398113, 9.84885780,
    10.04987562, 10.14889156, 10.34408043, 10.44030650, 10.63014581,
    11.26942766, 11.44552314, 11.70469991, 11.78982612, 12.20655561,
    12.28820572, 12.52996408, 12.76714533, 12.92284798, 13.15294643,
    13.37908816, 13.45362404, 13.82027496, 13.89244398, 14.03566884,
    14.10673597, 14.52583904, 14.93318452, 15.06651917, 15.13274595
])

# --------------------------------------------------------------
# PHYSICAL PARAMETERS
# --------------------------------------------------------------
tx = 1.0   # hopping in x
ty = 1.0   # hopping in y
# U  = 25.0  # on-site interaction strength

# Instead of fixed U, we define a sweep
U_min, U_max, dU = 0.0, 25.0, 1.0   # <-- choose your U range + step

# chemical potential scan
mu_min, mu_max, dmu = -5.0, 40.0, 0.2

# cluster geometry (2x2 plaquette)
xlen, ylen = 2, 2
m = xlen * ylen   # total number of sites = 4

nmax = 4          # maximum occupation per site

# Hilbert space dimension:
dim = (nmax+1)**m


def get_filename(tx, U_start, U_end, U_step, mu_start, mu_end, mu_step, suffix):
    """
    Generate a compact filename for the 2x2 cluster results.

    Example:
    phase_tx1.0_U(0.0-25.0)-(1.0)_Mu(0.0-40.0)-(0.2).csv
    """
    return (
        f"phase_tx{tx:.1f}_"
        f"U({U_start:.1f}-{U_end:.1f})-({U_step})_"
        f"Mu({mu_start:.1f}-{mu_end:.1f})-({mu_step})"
        f"{suffix}"
    )


# --------------------------------------------------------------
# BASIS GENERATION
# --------------------------------------------------------------
def basis_gen(n, k, min_elem, max_elem):
    allowed = range(max_elem, min_elem-1, -1)
    def helper(n, k, t):
        if k == 0:
            if n == 0:
                yield t
        elif k == 1:
            if n in allowed:
                yield t + (n,)
        elif min_elem * k <= n <= max_elem * k:
            for v in allowed:
                yield from helper(n - v, k - 1, t + (v,))
    return helper(n, k, ())

# --------------------------------------------------------------
# Binary search lookup
# --------------------------------------------------------------
def BinarySearch(lys, val):  
    first, last, index = 0, len(lys)-1, -1
    while (first <= last) and (index == -1):
        mid = (first+last)//2
        if lys[mid] == val:
            index = mid
        elif val < lys[mid]:
            last = mid -1
        else:
            first = mid +1
    return index

# --------------------------------------------------------------
# LATTICE STRUCTURE: 2x2 PLAQUETTE
# --------------------------------------------------------------
lattice = np.zeros((ylen,xlen),int)
lat_ind = 0
for yy in range(ylen):
    for xx in range(xlen):
        lattice[yy][xx] = lat_ind
        lat_ind += 1

row_pair = []
for yy in range(ylen):
    for xx in range(xlen-1):
        row_pair.append([lattice[yy][xx], lattice[yy][xx+1]])

col_pair = []
for xx in range(xlen):
    for yy in range(ylen-1):
        col_pair.append([lattice[yy][xx], lattice[yy+1][xx]])

# --------------------------------------------------------------
# BOUNDARY SITES
# --------------------------------------------------------------
lbs = [xlen * x for x in range(0, ylen)]
rbs = [(xlen * (x+1))-1 for x in range(0, ylen)]
tbs = [x for x in range(lbs[0], rbs[0]+1)]
bbs = [x for x in range(lbs[-1], rbs[-1]+1)]
all_bs = list(set().union(lbs, rbs, tbs, bbs))
lr_zip = list(zip(lbs, rbs))
tb_zip = list(zip(tbs, bbs))
bds = lr_zip + tb_zip
nds = len(all_bs)

# --------------------------------------------------------------
# BASIS CONSTRUCTION
# --------------------------------------------------------------
a = []
for n in range(0, (nmax*m)+1):
    for x in basis_gen(n, m, 0, nmax):
        a.append(x)
bscount = len(a)

T = np.zeros(dim)
for ii in range(dim):
    for jj in range(m):
        T[ii] += primesqt[jj]*float(a[ii][jj])

ind = np.argsort(T)
T   = sorted(T)

# --------------------------------------------------------------
# OPERATOR MATRICES
# --------------------------------------------------------------
a_mat_dic = {}
for ii in all_bs:
    row, col, val = [], [], []
    for vv in range(dim):
        Tg = 0
        if a[vv][ii]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiD = sqrt(float(ket[ii]+1.0))
            ket[ii] = ket[ii]+1
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiD)
    a_mat = (coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()).getH()
    a_mat_dic[ii] = a_mat

n_mat_dic = {}
for ii in range(m):
    row, col, val = [], [], []
    for vv in range(dim):
        if a[vv][ii] > 0:
            ket = np.copy(a[vv])
            diag = float(ket[ii])
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(diag)
    n_mat = coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()
    n_mat_dic[ii] = n_mat

# --------------------------------------------------------------
# HOPPING TERM (exact inside cluster)
# --------------------------------------------------------------
row, col, val = [], [], []
for ii in row_pair:
    for vv in range(dim):
        if a[vv][ii[1]] >= 1.0 and a[vv][ii[0]]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiDaj = -tx*sqrt(float((ket[ii[0]]+1.0)*ket[ii[1]]))
            ket[ii[0]] += 1; ket[ii[1]] -= 1
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiDaj)

for ii in col_pair:
    for vv in range(dim):
        if a[vv][ii[1]] >= 1.0 and a[vv][ii[0]]+1 < nmax+1:
            ket = np.copy(a[vv])
            aiDaj = -ty*sqrt(float((ket[ii[0]]+1.0)*ket[ii[1]]))
            ket[ii[0]] += 1; ket[ii[1]] -= 1
            Tg = 0
            for jj in range(m):
                Tg += float((primesqt[jj])*ket[jj])
            location = BinarySearch(T, Tg)
            uu = ind[location]
            row.append(uu); col.append(vv); val.append(aiDaj)

hop_mat = coo_matrix((val, (row, col)), shape=(dim, dim)).tocsr()
hop_mat = hop_mat + hop_mat.getH()

# --------------------------------------------------------------
# SELF-CONSISTENT MEAN FIELD LOOP
# --------------------------------------------------------------
phi, phi_new, diff, accuracy = {}, {}, {}, {}
gs = np.zeros(dim, dtype='float64')
for bsi in all_bs:
    accuracy[bsi] = 1e-6

phi_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()
psq_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()
int_mat = coo_matrix((dim, dim), dtype=np.double).tocsr()

# >>> storage additions >>>
results = []
# <<< storage additions <<<

#np.arange(start, stop, step) goes up to but not including stop.
#So if you write
# --- Parallel sweep over (U, mu) ---------------------------------------------

# Build grids (inclusive of endpoints, like your current arange + d step)
U_vals  = np.arange(U_min, U_max + dU, dU)
mu_vals = np.arange(mu_min, mu_max, dmu)

def solve_point(U, mu):
    """
    Self-consistent mean-field solve for a single (U, mu).
    Uses read-only globals: a_mat_dic, n_mat_dic, hop_mat, etc.
    Creates fresh local dicts to avoid race conditions.
    Returns one results row.
    """
    phi, phi_new, diff = {}, {}, {}
    for bsi in all_bs:
        phi[bsi] = 0.0
        phi_new[bsi] = 0.3
        diff[bsi] = abs(phi[bsi] - phi_new[bsi])

    iteration = 0
    # local working matrices (start as zeros)
    phi_mat = 0.0
    psq_mat = 0.0
    int_mat = 0.0

    while all(diff[bsi] > accuracy[bsi] for bsi in all_bs):
        H_mat = 0.0
        phi_mat = 0.0
        psq_mat = 0.0
        int_mat = 0.0

        # update phi from last iteration
        for bsi in all_bs:
            phi[bsi] = phi_new[bsi]

        # mean-field boundary couplings
        for xx in lr_zip:
            phi_mat += - tx*phi[xx[0]]*(a_mat_dic[xx[1]].getH() + a_mat_dic[xx[1]])
            phi_mat += - tx*phi[xx[1]]*(a_mat_dic[xx[0]].getH() + a_mat_dic[xx[0]])
            psq_mat += + 2.0*tx*phi[xx[0]]*phi[xx[1]]*eye(dim, dtype=np.double, format='csr')
        for yy in tb_zip:
            phi_mat += - ty*phi[yy[0]]*(a_mat_dic[yy[1]].getH() + a_mat_dic[yy[1]])
            phi_mat += - ty*phi[yy[1]]*(a_mat_dic[yy[0]].getH() + a_mat_dic[yy[0]])
            psq_mat += + 2.0*ty*phi[yy[0]]*phi[yy[1]]*eye(dim, dtype=np.double, format='csr')

        # on-site interaction + chemical potential
        for ii in range(m):
            int_mat += (U/2.0)*n_mat_dic[ii]*(n_mat_dic[ii]-eye(dim, dtype=np.double, format='csr'))
            int_mat += - mu*n_mat_dic[ii]

        H_mat = hop_mat + int_mat + psq_mat + phi_mat

        nev = min(dim, 3)
        if nev == 1:
            eigenvalues = H_mat[0, 0]
            eigenvectors = eye(dim, dtype=np.double, format='csr')
        else:
            eigenvalues, eigenvectors = eigsh(H_mat, k=3, which='SA')

        gs = eigenvectors[:, 0]

        for bsi in all_bs:
            phi_new[bsi] = gs.transpose().dot(a_mat_dic[bsi].dot(gs))
            diff[bsi] = abs(phi[bsi] - phi_new[bsi])

        iteration += 1
        if iteration > 10000:
            break

    rho = {}
    for bsi in all_bs:
        rho[bsi] = gs.transpose().dot(n_mat_dic[bsi].dot(gs))

    # print line (optional; printing from parallel workers can be noisy)
    # print(f"{tx:3.2f} {U:6.2f} {mu:6.2f} "
    #       f"{rho[0]:8.4f} {rho[1]:8.4f} {rho[2]:8.4f} {rho[3]:8.4f} "
    #       f"{phi_new[0]**2:8.4f} {phi_new[1]**2:8.4f} {phi_new[2]**2:8.4f} {phi_new[3]**2:8.4f} "
    #       f"{eigenvalues[0]:12.6f} {iteration:6d}")

    return [
        round(tx, 2), round(U, 2), round(mu, 2),
        round(rho[0], 4), round(rho[1], 4), round(rho[2], 4), round(rho[3], 4),
        round(phi_new[0]**2, 4), round(phi_new[1]**2, 4), round(phi_new[2]**2, 4), round(phi_new[3]**2, 4),
        round(float(eigenvalues[0]), 6), int(iteration)
    ]

# Optional: prevent BLAS oversubscription inside each process
os.environ.setdefault("OMP_NUM_THREADS", "1")

# Run grid in parallel
results = Parallel(n_jobs=-1, backend="loky", verbose=10)(
    delayed(solve_point)(U, mu)
    for U in U_vals
    for mu in mu_vals
)

# Keep rows ordered (U major, then mu)
results.sort(key=lambda r: (r[1], r[2]))

# --- Save CSV/NPZ with dynamic filenames ------------------------------------
csv_name = get_filename(tx,U_min, U_max, dU,mu_min,  mu_max, dmu, ".csv")
npz_name = get_filename(tx,U_min, U_max, dU,mu_min, mu_max, dmu, ".npz")


# --- Save CSV/NPZ exactly like before ----------------------------------------
headers = [
    "tx","U","mu",
    "rho0","rho1","rho2","rho3",
    "phi0_sq","phi1_sq","phi2_sq","phi3_sq",
    "E0","iters"
]
out_csv = Path(csv_name)
with out_csv.open("w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(results)

# Save NPZ
np.savez_compressed(
    npz_name,
    headers=np.array(headers, dtype=object),
    data=np.array(results, dtype=float)
)

print(f"\n✅ Saved results to {csv_name} and {npz_name}")


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 181 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 225 tasks      | elapsed:  


✅ Saved results to phase_tx1.0_U(0.0-25.0)-(1.0)_Mu(-5.0-40.0)-(0.2).csv and phase_tx1.0_U(0.0-25.0)-(1.0)_Mu(-5.0-40.0)-(0.2).npz


[Parallel(n_jobs=-1)]: Done 5850 out of 5850 | elapsed: 12.3min finished
